# **R181600B - SHEUNESU C. TAZVIVINGA**
# **R181558W - TAFADZWA N. YEMEKE**

**Question generator**

In [ ]:
!pip install -q streamlit

     |████████████████████████████████| 7.8MB 7.0MB/s 
     |████████████████████████████████| 174kB 43.6MB/s 
     |████████████████████████████████| 81kB 9.6MB/s 
     |████████████████████████████████| 4.2MB 41.1MB/s 
     |████████████████████████████████| 112kB 54.8MB/s 
     |████████████████████████████████| 71kB 9.4MB/s 
     |████████████████████████████████| 122kB 54.8MB/s 
ERROR: ipykernel 6.0.1 has requirement ipython>=7.23.1, but you'll have ipython 5.5.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 6.0.1 which is incompatible.


In [ ]:
!pip -q install pyngrok

     |████████████████████████████████| 747kB 6.9MB/s 


In [ ]:
from pyngrok import ngrok

In [ ]:
!ngrok authtoken 1npJviy2rm2H1RzUbevBUv2GLGw_ep9y73PeZMfA5zeSBzzs

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#!pip uninstall python-crfsuite

In [ ]:
!pip install python-crfsuite

     |████████████████████████████████| 747kB 6.8MB/s 


In [ ]:
!pip install sklearn-pycrfsuite

     |████████████████████████████████| 491kB 9.7MB/s 
  Created wheel for sklearn-pycrfsuite: filename=sklearn_pycrfsuite-0.4.0-py2.py3-none-any.whl size=11004 sha256=5f568a708cb7f141524a81f63cf0aec004b1eb525ad6ccc5f12abc9ec48cd78c
  Stored in directory: /root/.cache/pip/wheels/af/78/bd/189a8a1d7a6031ce1e1f8e81a5a8bdb0d3fba7e88c53f96107
  Created wheel for python-crfsuite-extension: filename=python_crfsuite_extension-0.9.7-cp37-cp37m-linux_x86_64.whl size=776116 sha256=4e5187553d6466e42d700e29dd74ae0143800f56c245e0e83ef4b5d2650b2df0
  Stored in directory: /root/.cache/pip/wheels/6d/ee/fa/10e4ae5dd9e71190ce1318fa7e2b82314bc880da1ec34ee567
Successfully built sklearn-pycrfsuite python-crfsuite-extension


In [151]:
%%writefile question_gen.py
import streamlit as st
import string
import pycrfsuite
import nltk
from nltk.corpus import stopwords
from nltk.tag import CRFTagger
from nltk import word_tokenize,sent_tokenize
import pandas as pd
import numpy as np
import spacy
import time


#Import PyTorch Framework
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

ct = CRFTagger()
nlp = spacy.load('en')

PAGE_CONFIG = {"page_title":"StColab.io", "page_icon":":smiley:","layout":"centered"}
st.set_page_config("PAGE_CONFIG")

st.sidebar.title("Dashboard")
st.title("Question Generator")

page = st.sidebar.selectbox("Choose action", ["Home", "Generate Questions"])
def main():
  if page == "Home":
    st.header("Choose from the options on the sidebar")
  if page == "Generate Questions":
    # Read in the train set data
    train_set  = pd.read_csv('/content/drive/My Drive/KBS/Question_Generator/Train_and_test_data/train.txt',sep=' ',names=['word','Brill','tag']).drop('Brill',1)
    train_set = [[tuple(x) for x in train_set.values]]

    # Read the Test data from txt
    sentence = ' '.join(open('/content/drive/My Drive/KBS/Question_Generator/Text_Passages2.txt','r').readlines()).rstrip("\n")

    def process_text(passage):
      exclude_set = set(['“','”',':'])

      no_punctuation = [char for char in passage if char not in string.punctuation + "“”."]
      no_punctuation = ''.join(no_punctuation)
      no_punctuation = [word for word in no_punctuation.split() if word.lower() not in stopwords.words('english')]
      
      return no_punctuation

    nltk.download('stopwords')
    #Removing stop words and punctuation
    sentence = process_text(sentence)
    sentence = ' '.join(sentence)  

    nltk.download('punkt')

    sentences = sent_tokenize(sentence)
    word_list = [[]]
    word_list.clear()
    wlist = []
    i = 0
    j = 0

    for sent in sentences:
        for word in sent.split():
            wlist.append(word)
            j += 1
            
        wlist.copy
        word_list.append(wlist.copy())
        wlist.clear()
        i += 1

    #Set the model of previously trained data set
    ct.set_model_file(model_file='/content/drive/My Drive/KBS/Question_Generator/model.crf.tagger')
    word_list = ct.tag_sents(word_list)

    test_set = pd.read_csv('/content/drive/My Drive/KBS/Question_Generator/Train_and_test_data/test.txt', sep=' ', names=['words','Brill','tag'], index_col=False).drop(['Brill','tag'],1)
    test_set = test_set.values.tolist()
    ct.tag_sents(test_set)

    test_setEval = pd.read_csv('/content/drive/My Drive/KBS/Question_Generator/Train_and_test_data/test.txt', sep=' ', names=['words','Brill','tag']).drop('Brill',1)
    test_setEval = [[tuple(x) for x in test_setEval.values]]
    #test the accuracy
    ct.evaluate(test_setEval)

    #Convert list of tuples to a numpy array
    word_list = np.array(word_list)
    word_list = np.reshape(word_list, (-1,2))

    # Find S -> NP VP NP
    # Returns if a subject is found
    # pattern variable = [pat0, pat1, pat2]
    def findSubject(pattern):
        if pattern[0] == 'NP' and pattern[1] == 'VP' and pattern[2] == 'NP':
            return True
        else:
            return False
    #Extract the Subject -> NP VP NP to find the subjects in a sentence
    #using Markov Chain Model

    pattern = []
    subjects = []
    word_list_length = len(word_list)

    for i in range(0, len(word_list)):
        #print(word_list[i][1].split('-')[1])
        try:
            if((word_list_length - i == 2)):
                break
            else:
                pattern = [word_list[i][1].split('-')[1], word_list[i+1][1].split('-')[1], word_list[i+2][1].split('-')[1]]
                # Call the function findSubject to identify potential subject elements and save them in an array
                if findSubject(pattern):
                    # If returns true consider 1st element as a potential Subject
                    print('Potential Subject Found at index {}'.format(i))
                    print('Subject -> {} => {} {}'.format(word_list[i][0], word_list[i+1][0], word_list[i+2][0]))
                    # Put the phrases in to a sentece and append it to an array.
                    sub = [word_list[i][0],word_list[i+1][0],word_list[i+2][0]]
                    subjects.append(' '.join(sub))
                    print('--------------------')
        # Exception handler
        except IndexError:
            print("Out of Index")
            break;
    # Find PERSONs in the filtered sentence using spaCy NER and build a DataFrame Object based on the data
    # DataFrame structure -> Word | POS_TAG | Person
    subjects = np.array(subjects)
    subjects = np.reshape(subjects, (-1,1))

    refferingDataFrame = pd.DataFrame(columns=('Word_POS-TAG_Person','nullColumn'))

    for i in range(0,len(subjects)):
        for j in range(0,len(subjects[i][0].split(" "))):
            wordBag = subjects[i][0].split(" ")
            pos = ct.tag([wordBag[j]])
            person = [ent.label_ for ent in nlp(pos[0][0]).ents]
            person = ' '.join(person)
            if not person:
                person = ""
                
            #Add a row to the DataFrame with the retrived data
            refferingDataFrame.loc[len(refferingDataFrame)] = [(pos[0][0] +"_"+ pos[0][1].split('-')[1] +"_"+ person).rstrip('_'), ""]

    # The RNN is created using PyTorch Framework. 
    #The RNN will clasify the Question according the the category
    lstm = nn.LSTM(3, 3)
    inputs = [autograd.Variable(torch.randn(1, 3)) for _ in range(5)]
    hidden = (autograd.Variable(torch.randn(1, 1, 3)), autograd.Variable(torch.randn(1, 1, 3)))

    for i in inputs:
        out, hidden = lstm(i.view(1, 1, -1), hidden)
        
    inputs = torch.cat(inputs).view(len(inputs), 1, -1)
    hidden = (autograd.Variable(torch.randn(1, 1, 3)), autograd.Variable(torch.randn(1, 1, 3)))
    out, hidden = lstm(inputs, hidden)
    #print(out)
    #print(hidden)

    def prepare_sequence(seq, to_ix):
        idxs = [to_ix[w] for w in seq]
        tensor = torch.LongTensor(idxs)
        return autograd.Variable(tensor)

    # Read training data from the csv
    # Trained data in the format => [(["Phrase"], ["Question"])]
    unprocessed_data = pd.read_csv('/content/drive/My Drive/KBS/Question_Generator/Train_and_test_data/LSTM_train_set.csv',header=None)

    # Transform the data in to a processable format
    t_data_for_lstm = []
    training_data = []
    for phrase in unprocessed_data.itertuples():
        t_data_for_lstm.append(list(zip([[phrase[1]]], [[phrase[2]]])))

    for i in range(0, len(t_data_for_lstm)):
        training_data.append(t_data_for_lstm[i][0])
        
    word_to_ix = {}
    tag_to_ix = {}

    for sent, tags in training_data:
        for word in sent:
            if word not in word_to_ix:
                word_to_ix[word] = len(word_to_ix)
        for word1 in tags:
            if word1 not in tag_to_ix:
                tag_to_ix[word1] = len(tag_to_ix)
                                
    EMBEDDING_DIM = 6
    HIDDEN_DIM = 6

    # The LSTM model
    class LSTMTagger(nn.Module):
        def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
            super(LSTMTagger, self).__init__()
            self.hidden_dim = hidden_dim
            
            self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
            self.lstm = nn.LSTM(embedding_dim, hidden_dim)
            self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
            self.hidden  = self.init_hidden()
            
        def init_hidden(self):
            return (autograd.Variable(torch.zeros(1, 1, self.hidden_dim)), autograd.Variable(torch.zeros(1, 1, self.hidden_dim)))
            
        def forward(self, sentence):
            embeds = self.word_embeddings(sentence)
            lstm_out, self.hidden = self.lstm(embeds.view(len(sentence), 1, -1), self.hidden)
            tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
            tag_scores = F.log_softmax(tag_space)
            return tag_scores
        
    # Setting the variables to Train the model
    model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
    loss_function = nn.NLLLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.1)

    #Values Before Training
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    t0 = time.time()
    iterations = 300
    progress_after = iterations/4

    # Training the RNN for 300 iterations
    for epoch in range(0,iterations):
        for sentence, tags in training_data:
            model.zero_grad()
            model.hidden = model.init_hidden()
            sentence_in = prepare_sequence(sentence, word_to_ix)
            targets = prepare_sequence(tags, tag_to_ix)
            # % of completion
            if(progress_after == epoch):
                st.write("Training {}% Completed".format((progress_after / iterations) * 100))
                progress_after = progress_after + (iterations / 4)
                
            tag_scores = model(sentence_in)
            
            loss = loss_function(tag_scores, targets)
            loss.backward()
            optimizer.step()
            
    st.write("Training Complete!\nTotal Training time :", round(time.time()-t0, 2), "s\n")

    #Values after training        
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    #print("Values After training")
    #print(tag_scores)

    st.markdown('**The questions are;**')
    # Formatting the result generated by the LSTM by adding the relevant Noun Phrases
    def generate_questions(model, ref_data_frame):
        question = ""
        sentence = ""
        index = 0
        for index, word_phrase in ref_data_frame.iterrows():
            # Predict the potentian sentence structure that can be used to generate the question.
            try:
                inputs = prepare_sequence([word_phrase[0]], word_to_ix)
                tag_scores = model(inputs)
            except KeyError:
                # ignore
                pass

            # Take the maximum probalilty, and based on the probabilty find index of value of the dictonary
            maxVal = max(tag_scores.data.numpy()[0])
            index_loc = 0
            probability_tag_scores = tag_scores.data.numpy().ravel()

            for i in range(0, len(probability_tag_scores)):
                if (probability_tag_scores.ravel()[i] == maxVal):
                    index_loc = i

            # Travers the dictonary and identify the key value based on the probability predicted
            for key, value in tag_to_ix.items():
                if (value == index_loc):
                    # Formatting the Key to generate a meaningful question
                    # Extracting subject and object of the tested sentence
                    sentence = word_phrase[0].split('_')
                    # Checking whether the question needs modification
                    if "NP" in sentence:
                        question = key.replace("NP", sentence[0])
                        st.write(question)
                    elif "VP" in sentence:
                        if "NP" in key and "N1" in key:
                            # Finding the complete sentence matching for the verb and index in subjects which matches for verb
                            index = [i for i,j in enumerate(subjects.ravel()) if sentence[0] in j]
                            ref_subject = subjects.ravel()[index][0].split()
                            # Replace for words NP and NP1
                            question = key.replace("NP", ref_subject[0]) 
                            question = question.replace("N1",ref_subject[-1])
                            st.write(question)
                            
                        else:
                            # If "VP" but only one "NP"
                            index = [i for i,j in enumerate(subjects.ravel()) if sentence[0] in j]
                            ref_subject = subjects.ravel()[index][0].split()
                            question = key.replace("NP", ref_subject[0])
                            st.write(question)

    generate_questions(model, refferingDataFrame)  

if __name__ == '__main__':
  main()

Overwriting question_gen.py


In [152]:
!streamlit run --server.port 80 question_gen.py&>/dev/null&

In [153]:
address = ngrok.connect(port=8501)

In [154]:
address

<NgrokTunnel: "http://9b2d6cdf4c8b.ngrok.io" -> "http://localhost:80">

**Run the code below when more than 2 tunnels are running**

In [150]:
ngrok.kill()